## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
from sklearn import preprocessing
import pandas as pd

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [579]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/ML AI/Neural Network and Deep Learning/prices.csv")

### Check all columns in the dataset

In [582]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(["date","symbol"], axis=1)

In [584]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data.loc[:999,:]
data["volume"] = data["volume"]/1000000

In [586]:
data.info

<bound method DataFrame.info of            open       close         low        high   volume
0    123.430000  125.839996  122.309998  126.250000   2.1636
1    125.239998  119.980003  119.940002  125.540001   2.3864
2    116.379997  114.949997  114.930000  119.739998   2.4895
3    115.480003  116.620003  113.500000  117.440002   2.0063
4    117.010002  114.970001  114.089996  117.330002   1.4086
5    115.510002  115.550003  114.500000  116.059998   1.0980
6    116.459999  112.849998  112.589996  117.070000   0.9496
7    113.510002  114.379997  110.050003  115.029999   0.7853
8    113.330002  112.529999  111.919998  114.879997   1.0937
9    113.660004  110.379997  109.870003  115.870003   1.5235
10   109.059998  109.300003  108.320000  111.599998   1.6539
11   109.730003  110.000000  108.320000  110.580002   0.9443
12   111.879997  111.949997  110.190002  112.949997   0.7449
13   111.320000  110.120003  110.000000  114.629997   0.7038
14   110.419998  111.000000  107.300003  111.400002  

### Divide the data into train and test sets

In [587]:
from sklearn.model_selection import train_test_split
X = data.drop(["volume"], axis=1)
y = data["volume"]
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
print(X)

           open       close         low        high
0    123.430000  125.839996  122.309998  126.250000
1    125.239998  119.980003  119.940002  125.540001
2    116.379997  114.949997  114.930000  119.739998
3    115.480003  116.620003  113.500000  117.440002
4    117.010002  114.970001  114.089996  117.330002
5    115.510002  115.550003  114.500000  116.059998
6    116.459999  112.849998  112.589996  117.070000
7    113.510002  114.379997  110.050003  115.029999
8    113.330002  112.529999  111.919998  114.879997
9    113.660004  110.379997  109.870003  115.870003
10   109.059998  109.300003  108.320000  111.599998
11   109.730003  110.000000  108.320000  110.580002
12   111.879997  111.949997  110.190002  112.949997
13   111.320000  110.120003  110.000000  114.629997
14   110.419998  111.000000  107.300003  111.400002
15   110.769997  110.709999  109.019997  112.570000
16   110.900002  112.580002  109.900002  112.970001
17   113.349998  114.470001  111.669998  114.589996
18   114.000

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn import preprocessing
#Normalize the data
#x_n = preprocessing.normalize(X_train)

  ## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")
x_n = tf.nn.l2_normalize(X_train,1)

2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(x_n,W), b, name='output')
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

3.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [594]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, y_:y_train})
    
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  305.429
Training loss at step:  5  is  282.72018
Training loss at step:  10  is  276.39557
Training loss at step:  15  is  274.6341
Training loss at step:  20  is  274.14352
Training loss at step:  25  is  274.0069
Training loss at step:  30  is  273.9688
Training loss at step:  35  is  273.95825
Training loss at step:  40  is  273.95526
Training loss at step:  45  is  273.95444
Training loss at step:  50  is  273.95422
Training loss at step:  55  is  273.95416
Training loss at step:  60  is  273.95416
Training loss at step:  65  is  273.95413
Training loss at step:  70  is  273.95416
Training loss at step:  75  is  273.95413
Training loss at step:  80  is  273.95416
Training loss at step:  85  is  273.95413
Training loss at step:  90  is  273.95413
Training loss at step:  95  is  273.95413


### Get the shapes and values of W and b

In [595]:
W

<tf.Variable 'Weights_26:0' shape=(4, 1) dtype=float32_ref>

In [596]:
b

<tf.Variable 'Bias_26:0' shape=(1,) dtype=float32_ref>

### Model Prediction on 1st Examples in Test Dataset

In [597]:
x_n = tf.nn.l2_normalize(X_test,1)
y = tf.add(tf.matmul(x_n,W), b, name='output')
print(y)

Tensor("output_31:0", shape=(300, 1), dtype=float32)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils import np_utils

In [0]:
irisdata = pd.read_csv("/content/drive/My Drive/ML AI/Neural Network and Deep Learning/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [600]:
irisdata = pd.get_dummies(irisdata, columns=["Species"])
irisdata.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = irisdata.drop(["Species_Iris-versicolor", "Species_Iris-virginica", "Species_Iris-setosa", "Id"], axis=1)
y = irisdata[["Species_Iris-versicolor", "Species_Iris-virginica", "Species_Iris-setosa"]]
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# create model
model = Sequential()
model.add(Dense(12,input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Model Training 

In [603]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=5)

Train on 105 samples, validate on 45 samples
Epoch 1/10
105/105 [==============================] - 1s 7ms/step - loss: 0.6634 - acc: 0.6667 - val_loss: 0.6116 - val_acc: 0.6667
Epoch 2/10
105/105 [==============================] - 0s 1ms/step - loss: 0.5980 - acc: 0.6667 - val_loss: 0.5578 - val_acc: 0.6667
Epoch 3/10
105/105 [==============================] - 0s 996us/step - loss: 0.5427 - acc: 0.6730 - val_loss: 0.5134 - val_acc: 0.6963
Epoch 4/10
105/105 [==============================] - 0s 982us/step - loss: 0.4911 - acc: 0.7746 - val_loss: 0.4741 - val_acc: 0.8296
Epoch 5/10
105/105 [==============================] - 0s 1ms/step - loss: 0.4445 - acc: 0.8730 - val_loss: 0.4356 - val_acc: 0.8593
Epoch 6/10
105/105 [==============================] - 0s 972us/step - loss: 0.3997 - acc: 0.8857 - val_loss: 0.4034 - val_acc: 0.8667
Epoch 7/10
105/105 [==============================] - 0s 961us/step - loss: 0.3629 - acc: 0.8825 - val_loss: 0.3800 - val_acc: 0.8667
Epoch 8/10
105/105 [===

### Model Prediction

In [604]:
y_predict=model.predict_classes(X_test)
print(y_predict)

[1 1 2 1 1 2 0 1 2 0 1 0 2 1 2 1 1 1 2 2 0 1 1 1 1 1 0 0 1 1 1 0 2 1 0 2 2
 2 2 1 1 1 1 1 0]


### Save the Model

In [0]:
from keras.models import load_model

model.save('/content/drive/My Drive/ML AI/Neural Network and Deep Learning/iris_pred_keras')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?